In [ ]:
from dep_tools.loaders import OdcLoader
from dep_tools.searchers import PystacSearcher
from odc.stac import configure_s3_access
from ldn.geomad import (
    GeoMADLandsatProcessor,
    USGS_CATALOG,
    USGS_COLLECTION
)
from dask.distributed import Client as DaskClient

from ldn.grids import get_gridspec

In [ ]:
# Reload functions
%load_ext autoreload
%autoreload 2

In [ ]:
# Make sure we can access S3
_ = configure_s3_access(cloud_defaults=True, requester_pays=True)

In [ ]:
from odc.stac import configure_s3_access, configure_rio, load

import os

# TODO: Remove these hardcoded secrets before committing
# Set AWS credentials
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_SESSION_TOKEN"] = ""

# Print to verify
print("AWS_ACCESS_KEY_ID:", os.environ.get("AWS_ACCESS_KEY_ID"))
print("AWS_SECRET_ACCESS_KEY:", os.environ.get("AWS_SECRET_ACCESS_KEY"))
print("AWS_SESSION_TOKEN:", os.environ.get("AWS_SESSION_TOKEN"))

os.environ["AWS_REQUEST_PAYER"] = "requester"
print("AWS_REQUEST_PAYER:", os.environ.get("AWS_REQUEST_PAYER"))

# Make sure we can access S3
_ = configure_s3_access(cloud_defaults=True, requester_pays=True, aws_unsigned=False)

configure_rio(
    cloud_defaults=True,
    # aws={"aws_unsigned": True, "aws_request_payer": "requester"},
)

# Define years and tiles to test
# years = "2000,2005,2010,2015,2020,2025"
years = "2010,2015,2020,2025"
years_list = years.split(",")

grid_tiles = [
    "53_113", # Pacific island (atolls). Kiribati.
    "413_86", # Pacific islands (volcanic). Fiji.
    "29_86", # Fiji antimeridian crossing. Fiji.
    "127_134", # Caribbean (atolls). Belize.
    "162_117", # Carribean (land). Suriname.
    "197_133", # Africa. Cape Verde.
    "268_94", # Indian Ocean. Comoros.
]

In [ ]:
import pystac_client

from ldn.geomad import USGS_CATALOG, USGS_COLLECTION
# catalog = pystac_client.Client.open(http_to_s3_url("https://landsatlook.usgs.gov/stac-server"))
catalog = pystac_client.Client.open("https://landsatlook.usgs.gov/stac-server")
# catalog = pystac_client.Client.open(USGS_CATALOG)


# Set a bounding box
# [xmin, ymin, xmax, ymax] in latitude and longitude
bbox = [-179.78797886784525, -17.969338895119208, -179.3186512262919, -17.252353456473642]

# Set a start and end date
start_date = "2024-01-01"
end_date = "2024-12-31"

# Set product ID as the STAC "collection"
collections = ["landsat-c2l2-sr"]
# collections = [USGS_COLLECTION]
catalog

In [ ]:
# Build a query with the parameters above
query = catalog.search(
    bbox=bbox,
    collections=collections,
    query={"landsat:collection_category": {"eq": "T1"}, "landsat:cloud_cover_land": {"lt": 50.00}},
    datetime=f"{start_date}/{end_date}",
)

# Search the STAC catalog for all items matching the query
items = list(query.items())
print(f"Found: {len(items):d} T1 datasets")

print(items[0])
print(items[0].assets['red'])

In [ ]:
# from ldn.geomad import http_to_s3_url

def http_to_s3_url(http_url: str) -> str:
    # import pdb; pdb.set_trace()
    # set_trace()
    """Convert a USGS HTTP URL to an S3 URL"""
    s3_url = http_url.replace(
        "https://landsatlook.usgs.gov/data", "s3://usgs-landsat"
    ).rstrip(":1")
    print(f"Converted {http_url} to {s3_url}")
    return s3_url

ds = load(
    items,
    bands=["red", "green", "blue"],
    # bands=["red"],
    crs="utm",
    resolution=30,
    bbox=bbox,
    skip_broken_datasets=False,
    fail_on_error=True,
    # chunks={"x": 512, "y": 512}, # This makes loading lazy
    patch_url=http_to_s3_url,
    # groupby="solar_day",
)

ds



In [ ]:
# import boto3
# s3 = boto3.client("s3")
# s3.head_object(Bucket="usgs-landsat", Key="collection02/level-2/standard/oli-tirs/2024/073/072/LC08_L2SR_073072_20241002_20241008_02_T1/LC08_L2SR_073072_20241002_20241008_02_T1_SR_B4.TIF", RequestPayer="requester")

In [ ]:
ds.to_array().plot.imshow(col="time", col_wrap=4, robust=True, size=5)

In [ ]:
best_image_date = "2024-06-04"

best_image = ds.sel(time=best_image_date).squeeze()

best_image

In [ ]:
visualisation = best_image.odc.to_rgba()

visualisation.odc.explore()

In [ ]:
percentile_stretch = (1, 95)

rgb_array = best_image.to_array().values

stretch_vmin, stretch_vmax = np.nanpercentile(rgb_array, percentile_stretch)

stretch_visualisation = best_image.odc.to_rgba(vmin=stretch_vmin, vmax=stretch_vmax)

stretch_visualisation.odc.explore()